Notebook for consolidated results. Path might need to be changed. The parquet file I have for 2006-2017 data is too large to upload to github, so I include a google drive link for it here. It is just the original data file downloaded from web, preprocessing is done with function in this notebook, so if you already have the csv file you can just change the code about read parquet to read csv

https://drive.google.com/file/d/1YOUE6DZBP445NEfzbOi907emhREZKUQr/view?usp=sharing


In [2]:
from google.colab import drive
import os 
drive.mount('/gdrive/')

Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
! pip install PyAstronomy

     |████████████████████████████████| 727 kB 14.7 MB/s 
     |████████████████████████████████| 85 kB 3.1 MB/s 
  Created wheel for PyAstronomy: filename=PyAstronomy-0.17.0-py3-none-any.whl size=522050 sha256=2b424c08a91207b77d28b2e1f7c4a714d881b3d142fcaa8a1fb5d5be019313ae
  Stored in directory: /root/.cache/pip/wheels/10/f4/cc/fe117c538c81443a6ba0e852ee8d69866a08e5163d2050aae5
  Created wheel for quantities: filename=quantities-0.12.5-py3-none-any.whl size=80135 sha256=bac5c0fec37e8d3623600fa26077f810974e875348721ee5e3da05cbfbdc96a5
  Stored in directory: /root/.cache/pip/wheels/67/e7/32/0bb6d5bd0f619e583b6f1f4c710b535df898a1083e1e5d066c
Successfully built PyAstronomy quantities


In [1]:
import pandas as pd
import time
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from PyAstronomy import pyasl
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import statsmodels.api as sm
from sklearn.metrics import classification_report, roc_curve, auc

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# read file (traffic.parquet contains all the records, can be found in Github)
# please change data_path
data_path = '/gdrive/MyDrive/traffic_stop/traffic.parquet'
data = pd.read_parquet(data_path, engine = 'pyarrow')

### Preprocessing: This is actually the step when I run out of RAM

In [3]:
keep_col = ['date', 
                'time', 
                'county_name', 
                'subject_race',
                'subject_sex',
                'violation', 
                'citation_issued',
                'contraband_found',
                'contraband_drugs',
                'contraband_weapons',
                'search_conducted',
                'search_vehicle',
                ]
rm_na_col = ['county_name','violation','subject_race','subject_sex']

In [4]:
def remove_empty_rows(df, colName):
	df = df[df[colName] != 'unknown']
	df = df[df[colName].notna()]
	return(df)
 
def exclusive(vio_lst):
  for vio in vio_lst:
    # if there is any violation other than speeding, return 0
    if 'speed' not in vio:
      return 0
  # if all of the violations (for this record) include speeding, return 1
  return 1

def preprocess(df, keep_col, rm_na_col, county_path, violation_type = None):

  start = time.time()
  # drop unrelated columns
  col_drop = [col for col in df.columns if col not in keep_col]
  df.drop(col_drop, axis = 1, inplace = True)

  # remove rows with missing values in rm_na_col
  for col in rm_na_col:
    df = remove_empty_rows(df, col)

  # convert violation to lower cases
  df['violation'] = [s.lower() for s in df['violation']]

  # if we only want to include certain violation type:
  if violation_type == 'speed':
    df = df.loc[df['violation'].str.contains('speed', regex=False),:]

  # if we only want to include cases with no violations other than speeding
  if violation_type == 'speed_exclusive':
    df = df.loc[df['violation'].str.contains('speed', regex = False),:]
    df['violation'] = df['violation'].map(lambda x: x.replace('(#)',''))
    df['violation'] = df['violation'].map(lambda x: x.strip())
    # get a list of violations for each record, and apply self-defined func exclusive
    df['all_violation'] = df['violation'].str.split('|')
    df['speeding_only'] = df['all_violation'].map(lambda x: exclusive(x))
    # filter out rows with violations other than speeding
    df = df.loc[df['speeding_only'] == 1,:]
    df.drop(['speeding_only','all_violation'], axis = 1, inplace = True)

  # after selecting based on violation, drop the column
  df.drop('violation', 1, inplace=True)

  # adding time of year info
  df['year'] = pd.to_datetime(df['date']).dt.year
  df['yearfrac'] = [pyasl.decimalYear(d) for d in pd.to_datetime(df['date'])]
  df['yearfrac'] = df['yearfrac'] - df['year']
  df['minute'] = df['time'].map(lambda x: x.minute)
  df['hour'] = df['time'].map(lambda x: x.hour)
  df['time'] = df['hour'] + df['minute'] / 60
  scaler = MinMaxScaler()
  scaler.fit(np.array(df['time']).reshape(-1,1))
  df['time'] = scaler.transform(np.array(df['time']).reshape(-1,1))
  df.drop(['hour', 'minute', 'date', 'year'], 1, inplace=True)
        
  # County names are converted to county type - metropolitan, micropolitan or non-core
  # For definitions, see US OMB website

  # read in county info csv
  # Please change the file path !!!!!
  county_df = pd.read_csv(county_path)
  county_df = county_df[county_df['State']=='Texas']
  county_df = county_df.filter(items=['Metropolitan Status', 'County Name'])

  # transform column
  df['county_name'] = [name[:-7] for name in df['county_name']]
  df = df.join(county_df.set_index('County Name'), on='county_name')
  df.drop('county_name', 1, inplace=True)
  df.rename(columns={'Metropolitan Status':'county_type'}, inplace=True)

  # Convert citation issued and warning issued columns to integer
  df = df.astype({'citation_issued': 'int64'})

  # search and contraband related variables have three levels: None, True, False, if not True, use 0, else 1
  for col in ['contraband_found','contraband_drugs','contraband_weapons','search_conducted','search_vehicle']:
    df[col] = df[col].map({True: 1, False: 0, None: 0})

  # if race is 'other'/'unknown', make them one level other/unknown
  # df['subject_race'] = df['subject_race'].replace({'other':'other/unknown','unknown':'other/unknown'})
  # update: if race is other/unknown, we delete the rows!
  df = df.loc[(df['subject_race'] != 'unknown') & (df['subject_race'] != 'other'),:]
  #print(df['subject_race'].value_counts())
  # get dummies for race and sex
  df = pd.get_dummies(df)
 
  # Base level: White, Male, Non_core (county type)
  df.drop(['subject_race_white', 'subject_sex_male', 'county_type_Non core','subject_race_other','subject_race_unknown'], axis = 1, inplace = True)
  print(df.columns)
  print('preprocessing time: %d'%(time.time()-start))
  return df 

In [24]:
col_lst = ['time', 'citation_issued', 'contraband_found', 'contraband_drugs', 
           'contraband_weapons', 'search_conducted', 'search_vehicle', 'yearfrac',
           'subject_race_asian/pacific islander', 'subject_race_black', 'subject_race_hispanic', 'subject_sex_female', 
           'county_type_Metropolitan', 'county_type_Micropolitan']

### Run Analysis for 3 different kinds of violations (all, with speeding, only speeding)

In [25]:
num_feat = len(col_lst)
# this df is later pass into run_year_analysis to get all estimates
df_all = pd.DataFrame({'variable':col_lst})

# keep track of coefficient estimates and evaluation metrics
asianpacific = []	
black = []	
hispanic = []	
roc_auc = []

vio_type = ['None','speed','speed_exclusive']
def run_analysis(data, output_path, keep_col, rm_na_col, df_all, county_path, violation_type = vio_type):
    """
    Input:
    data_path: the folder path where all the yearly-based parquet files are saved

    output_path: output path for csv files -> not used yet, because I want to print out the result and directly 
    save it afterwards. After we make sure there is no problem about how I did the analysis, we integrate this part
    into the function

    keep_col, rm_na_col,violation_type: parameters for preprocessing
    df_years: an empty data frame where we can save the logistic regression results

    Output:
    A result dataframe
    """
    for vio in vio_type:
      df = preprocess(df = data,keep_col = keep_col, rm_na_col = rm_na_col, county_path = county_path, violation_type = vio)
      # train test split
      y = df['citation_issued']
      X = df.drop('citation_issued', axis = 1)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
      # note that in some columns, we could have all 0 values, this makes the matrix singular and cannot run logistic regression
      # Those columns also contain no useful information. So, we drop those columns
      for col in X.columns:
        if len(X_train.loc[X_train[col] == 0,:]) == len(X_train):
          X_train.drop(col, axis = 1, inplace = True)
          X_test.drop(col, axis = 1, inplace = True)
          print(vio, ': ','Delete ', col)

      #adding constant to X
      X_train_with_constant = sm.add_constant(X_train)
      X_test_with_constant = sm.add_constant(X_test)
      # building the model and fitting the data
      log_reg = sm.Logit(y_train, X_train_with_constant).fit()
          
      res_df = pd.DataFrame({'variable':list(log_reg.params.index), 'coef':list(log_reg.params.values), 'odds_ratio':list(np.exp(log_reg.params.values)), 'pvalue':list(log_reg.pvalues)})
      df_all = df_all.merge(res_df, left_on = 'variable', right_on = 'variable', how = 'left')
      df_all = df_all.rename(columns = {'coef': ('coef_' + str(vio)), 'odds_ratio': ('odds_ratio_' + str(vio)), 'pvalue': ('pvalue_' + str(vio))})

      # append race coefs to result list for plotting
      black.append(log_reg.params['subject_race_black'])
      hispanic.append(log_reg.params['subject_race_hispanic'])
      asianpacific.append(log_reg.params['subject_race_asian/pacific islander'])

      # for test set evaluation
      y_pred = log_reg.predict(X_test_with_constant)
      fpr, tpr, threshold = roc_curve(y_test, y_pred)
      roc_auc.append(auc(fpr, tpr))
      # for classification report, choose threshold 0.5
      y_pred_c = [1 if y > 0.5 else 0 for y in y_pred]
      print('Violation: ', vio )
      print('-----------------------------------------------')
      print(classification_report(y_test, y_pred_c))
      print('-----------------------------------------------')

    return df_all

      # write result df to csv file


In [12]:
# county csv file path
county_path = '/gdrive/MyDrive/traffic_stop/2014-2018.csv'

In [ ]:
res_df = run_analysis(data, output_path = ' ', keep_col = keep_col, rm_na_col = rm_na_col, county_path = county_path, df_all = df_all)

In [28]:
# res_df should look something like this (this table is obtained from smaller data, stats not valid) --- NaN means that variable not included in analysis because it only has a single value.
res_df

,variable,coef_None,odds_ratio_None,pvalue_None,coef_speed,odds_ratio_speed,pvalue_speed,coef_speed_exclusive,odds_ratio_speed_exclusive,pvalue_speed_exclusive
0,time,-0.265693,0.766674,0.000000e+00,-0.089557,0.914336,5.635671e-22,-0.143230,0.866555,8.607800e-34
1,citation_issued,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,contraband_found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,contraband_drugs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,contraband_weapons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,search_conducted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,search_vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,yearfrac,-0.118982,0.887824,8.864939e-103,-0.097669,0.906949,1.302140e-40,-0.106416,0.899051,1.347093e-31
8,subject_race_asian/pacific islander,0.229005,1.257349,1.124417e-73,0.532300,1.702844,2.198353e-265,0.635356,1.887694,2.220594e-288
9,subject_race_black,0.485210,1.624517,0.000000e+00,0.508686,1.663105,0.000000e+00,0.259997,1.296927,3.510297e-177


In [ ]:
# print odds ratios
asianpacific = np.exp(asianpacific)
black = np.exp(black)
hispanic = np.exp(hispanic)
print(asianpacific)
print(black)
print(hispanic)
print(roc_auc)